# TDAmeritrade's API to get Historical EOD data notes
From script 1 and script 2, we will get the latest stock list, which contians all the instrusments we are interested to get data for.
For retriving data, the real question is about stock split.


In [22]:
import time_utilities
import pandas as pd
import numpy as np
import datetime
import os
import pytz
import TD_API
import Paths
import time
import api_keys
import requests
import utilities

In [13]:
time_utilities.eastern_to_epoch('1970-01-01 01:00:00')

21600000

In [23]:
# We first need a function which will get all eod data up to today. This function will run at the first time, and will run every time a split happens.


def get_eod_data_from_start(symbol):
	#Define payload
	payload = {
		'apikey': api_keys.APIKEY_DAILY,
		'periodType' : 'ytd',
        'period' : '1',
        'frequencyType': 'daily',
        'frequency':'1',
		'startDate': '21600000' #This is 1970-01-01
	}
	endpoint = TD_API.get_price_history_endpoint(symbol)

	#Get data
	try:
		time.sleep(0.6)
		td_response = requests.get(url=endpoint, params=payload)
		content = td_response.json()
		df = pd.DataFrame(content['candles'])
		df.to_csv(os.path.join(updates_folder, symbol + '.csv'), index=False)
		return None
	except:
		print(symbol + " failed.")
		return symbol



In [24]:
today = time.strftime("%Y-%m-%d")
updates_folder = os.path.join(Paths.Daily_Data_Update_Folder, today)
if not os.path.exists(updates_folder):
	os.makedirs(updates_folder)

In [25]:
from tqdm import tqdm
stock_list = utilities.get_stock_list()
failed_list = []
progress_bar = tqdm(stock_list)
for symbol in progress_bar:
	# Show progress of the loop
	progress_bar.set_description("Requesting %s" % symbol)
	failed = get_eod_data_from_start(symbol)
	if failed is not None:
		failed_list.append(failed)

if len(failed_list) > 0:
	with open(os.path.join(Paths.Daily_Data_Failed_List, today + '.txt'), 'w') as f:
		for symbol in failed_list:
			f.write(symbol + '\n')



Requesting AC:   1%|          | 92/14544 [01:46<4:37:57,  1.15s/it] 